---
# 0. 라이브러리 설치 및 불러오기
----

In [14]:
!pip install transformers
!pip install datasets
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=870793bba700acda3607fde4cd7fabd9297fd414917a45a1192b6834c5f50dc6
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
from torchvision import transforms, datasets

from PIL import Image
from pprint import pprint
import easydict

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') # GPU 할당

import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd


---
# 1. Data
---

In [15]:
from datasets import load_dataset

sts_train = load_dataset("mteb/sts13-sts" ,split="test[0%:80%]")
sts_test = load_dataset("mteb/sts13-sts" ,split="test[80%:100%]")
# sts = load_dataset("mteb/sts13-sts")

nli = load_dataset("multi_nli")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/mteb___json/mteb--sts13-sts-a3d8d9f11c3fb4fb/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Dataset multi_nli downloaded and prepared to /root/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
sts_train

Dataset({
    features: ['split', 'sentence1', 'sentence2', 'score'],
    num_rows: 1200
})

In [ ]:
nli

DatasetDict({
    train: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9832
    })
})

In [ ]:
nli['train'][0]['premise'], nli['train'][0]['hypothesis'], nli['train'][0]['label'] 

('Conceptually cream skimming has two basic dimensions - product and geography.',
 'Product and geography are what make cream skimming work. ',
 1)

---
# 2. Pre-Trained Model
---

In [2]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model = model.to(device)

from sentence_transformers import SentenceTransformer
sbert = SentenceTransformer('all-MiniLM-L6-v2')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


---
# 3. Dataset & DataLoader
---

In [ ]:

class SBertDataset(Dataset):
  def __init__(self, data):
    self.sentence1 = []
    self.sentence2 = []
    self.label = []
    for meta in data:
      self.sentence1.append(meta['sentence1'])
      self.sentence2.append(meta['sentence2'])
      self.label.append(meta['score'])

  def __len__(self):
    return len(self.sentence1)

  def __getitem__(self, idx):
    return self.sentence1[idx], self.sentence2[idx], self.label[idx]

In [ ]:
class BertDataset(Dataset):
  def __init__(self, data, tokenizer):
    self.sentence1 = []
    self.sentence2 = []
    self.label = []
    self.tokenizer = tokenizer
    for meta in data:
      # BERT -> max 512 length
      self.sentence1.append(self.tokenizer(meta['sentence1'], max_length=128, padding='max_length', return_tensors='pt'))
      self.sentence2.append(self.tokenizer(meta['sentence2'], max_length=128, padding='max_length', return_tensors='pt'))
      self.label.append(meta['score'])

  def __len__(self):
    return len(self.sentence1)

  def __getitem__(self, idx):
    return self.sentence1[idx], self.sentence2[idx], self.label[idx]

In [ ]:
Batch_Size = 16

train_data = BertDataset(sts_train, tokenizer)
valid_data = BertDataset(sts_test, tokenizer)
train_loader = DataLoader(train_data, batch_size=Batch_Size, shuffle=True, drop_last = True)
valid_loader = DataLoader(valid_data, batch_size=Batch_Size, shuffle=False, drop_last = True)

In [ ]:
print(len(train_loader.dataset))
print(len(valid_loader.dataset))

1200
300


In [ ]:
train_data.__getitem__(7)

({'input_ids': tensor([[  101,  1037,  9495, 12005, 26243,  2015,  1037,  3433,  1012,  1996,
           3433,  2089,  2593,  2022,  2070,  2785,  1997,  2895,  3214,  2000,
           4769,  1996,  9495,  2724,  2030,  2019,  5228,  8689,  1997,  1996,
           9495,  2724,  1012,   102,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,   

---
# 4. Model
---

---
### 4-1. Bert Model
----

In [ ]:
## BERT STS (cosine similarity)
epochs = 20

optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

In [ ]:
for i in train_loader:
  sents1, sents2, labels = i
  print(sents1['input_ids'].shape)
  print(sents1['token_type_ids'].shape)
  print(sents1['attention_mask'].shape)
  print(labels.shape)
  break

torch.Size([16, 1, 128])
torch.Size([16, 1, 128])
torch.Size([16, 1, 128])
torch.Size([16])


In [ ]:
for epoch in range(1, epochs+1):
  train_loss = 0.0
  cor = 0.0
  tot = 0.0
  for i, batch_data in enumerate(tqdm(train_loader)):
      model.train()
      optimizer.zero_grad()
      sents1, sents2, labels = batch_data
      sents1 = sents1.to(device)
      sents2 = sents2.to(device)
      labels = labels.to(device)
      
      ###### CLS Token Embedding 구하기 ######
      # [batch_size, max_length, 768]
      sents1_embeddings = model(sents1['input_ids'].squeeze(1), sents1['attention_mask'].squeeze(1), sents1['token_type_ids'].squeeze(1))[0][:,0,:]
      sents2_embeddings = model(sents2['input_ids'].squeeze(1), sents2['attention_mask'].squeeze(1), sents2['token_type_ids'].squeeze(1))[0][:,0,:]
      # MSE loss
                    ##### dot-product 구하기 #####
      loss = (((sents1_embeddings*sents2_embeddings).sum(dim=-1) - labels)**2).sum()
      train_loss += loss.item()
      loss.backward()
      optimizer.step()
      
  print(f'EPOCH_{epoch}_LOSS',train_loss / len(train_loader.dataset))
  for i, batch_data in enumerate(tqdm(valid_loader)):
      model.eval()
      with torch.no_grad():
        sents1, sents2, labels = batch_data
        sents1 = sents1.to(device)
        sents2 = sents2.to(device)
        labels = labels.to(device)

        sents1_embeddings = model(sents1['input_ids'].squeeze(1), sents1['attention_mask'].squeeze(1), sents1['token_type_ids'].squeeze(1))[0][:,0,:]
        sents2_embeddings = model(sents2['input_ids'].squeeze(1), sents2['attention_mask'].squeeze(1), sents2['token_type_ids'].squeeze(1))[0][:,0,:]
        sim = (sents1_embeddings*sents2_embeddings).sum(dim=-1)
        cor += np.correlate(sim.cpu().numpy(), labels.cpu().numpy())
        tot += len(sents1)
  print('#'*50)
  print (f'EPOCH_{epoch}:', cor/tot)
  print('#'*50)

  0%|          | 0/75 [00:00<?, ?it/s]

3938.3654477339323


  0%|          | 0/18 [00:00<?, ?it/s]

##################################################
[741.45786066]
##################################################


  0%|          | 0/75 [00:00<?, ?it/s]

...

##################################################
[449.5160705]
##################################################


  0%|          | 0/75 [00:00<?, ?it/s]

519.0467441820126


  0%|          | 0/18 [00:00<?, ?it/s]

##################################################
[338.52433216]
##################################################


---
### 4-2. Sentence BERT Model
----

In [ ]:
train_data = SBertDataset(sts_train)
valid_data = SBertDataset(sts_test)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, drop_last = True)
valid_loader = DataLoader(valid_data, batch_size=64, shuffle=False, drop_last = True)

In [ ]:
## SBERT STS (cosine similarity)
epochs = 20
sbert = sbert.to(device)
optimizer = torch.optim.Adam(sbert.parameters(), lr=1e-5)


In [ ]:
train_data.__getitem__(7)

('a trigger elicits a response. the response may either be some kind of action meant to address the trigger event or an expressed judgment of the trigger event .',
 'meet by design ; be present at the arrival of ;',
 0.0)

In [ ]:
sbert.encode(['a trigger elicits a response. the response may either be some kind of action meant to address the trigger event or an expressed judgment of the trigger event .', 'hihihihii']).shape

(2, 384)

In [ ]:
for epoch in range(1, epochs+1):
  cor = 0.0
  tot = 0.0
  total_loss = 0.0
  for i, batch_data in enumerate(tqdm(train_loader)):
      sbert.train()
      optimizer.zero_grad()
      sents1, sents2, labels = batch_data

      sents1_embeddings = torch.tensor(sbert.encode(sents1), dtype=torch.float, requires_grad=True)
      sents2_embeddings = torch.tensor(sbert.encode(sents2), dtype=torch.float, requires_grad=True)
      
      # MSE loss
      loss = (((sents1_embeddings*sents2_embeddings).sum(dim=-1) - labels)**2).sum()
      total_loss += loss.item()

      loss.backward()
    
      optimizer.step()
  print(f'EPOCH_{epoch}_LOSS',train_loss / len(train_loader.dataset))

  for i, batch_data in enumerate(tqdm(valid_loader)):
      sbert.eval()
      with torch.no_grad():
        sents1, sents2, labels = batch_data

        sents1_embeddings = torch.tensor(sbert.encode(sents1), dtype=torch.float)
        sents2_embeddings = torch.tensor(sbert.encode(sents2), dtype=torch.float)

        sim = (sents1_embeddings*sents2_embeddings).sum(dim=-1)
        cor += np.correlate(sim.cpu().numpy(), labels.cpu().numpy())
        tot += len(sents1)
  
  print('#'*50)
  print (f'EPOCH_{epoch}:', cor/tot)
  print('#'*50)

  0%|          | 0/18 [00:00<?, ?it/s]

EPOCH_1_LOSS 519.0467441820126


  0%|          | 0/4 [00:00<?, ?it/s]

##################################################
EPOCH_1: [1.67360037]
##################################################


  0%|          | 0/18 [00:00<?, ?it/s]

...

##################################################
EPOCH_19: [1.67360037]
##################################################


  0%|          | 0/18 [00:00<?, ?it/s]

EPOCH_20_LOSS 519.0467441820126


  0%|          | 0/4 [00:00<?, ?it/s]

##################################################
EPOCH_20: [1.67360037]
##################################################


---
### 4-3. NLI Model
----

- 문장 2개를 모델에 넣고 2개 문장에 대한 관계를 맞추는 TASK

In [ ]:
class SentenceClassificationModelNLI(nn.Module):
  def __init__(self):
    super(SentenceClassificationModelNLI, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    # CLS token x class nums
    self.classification_layer = nn.Linear(768, 3)
  
  def forward(self, input_ids, attention_mask, token_type_ids):
    bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    cls_token = bert_output[0][:,0,:]
    pred = self.classification_layer(cls_token)
    return pred

In [ ]:
class BertDataset(Dataset):
  def __init__(self, data, tokenizer):
    self.sentence = []
    self.label = []
    self.tokenizer = tokenizer
    for meta in data:
      self.sentence.append(self.tokenizer(meta['premise'], meta['hypothesis'],  max_length=256, padding='max_length', truncation=True, return_tensors='pt'))
      self.label.append(meta['label'])

  def __len__(self):
    return len(self.sentence)

  def __getitem__(self, idx):
    return self.sentence[idx], self.label[idx]

In [ ]:
## NLI (multi-class classification)
train_data = BertDataset(nli['validation_matched'], tokenizer)
valid_data = BertDataset(nli['validation_matched'], tokenizer)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last = True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False, drop_last = True)


In [ ]:
epochs = 20
model = SentenceClassificationModelNLI().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
loss_func = nn.CrossEntropyLoss()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_data.__getitem__(7)

({'input_ids': tensor([[  101, 13419,  3849,  2000,  2022,  1996,  2069,  2060,  2537,  2415,
           2383,  2151,  3653, 29048,  2015,  2000,  6018, 14842,  2012,  2035,
           1010,  2021, 18527,  2017,  1005,  2128,  2941,  2062,  3497,  2000,
           2156,  1996,  2573,  1997,  2938,  3148, 18902,  4097,  2030, 27011,
          12032, 12411,  3491,  1999,  2885,  2030,  2167,  2637,  2084,  1999,
           2634,  2993,  1012,   102,  2087,  1997, 27011, 12032, 12411,  1005,
           1055,  2147,  2064,  2022,  2179,  1999,  2647,  6407,  1012,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,   

In [ ]:
for epoch in range(1, epochs+1):
  acc = 0.0
  tot = 0.0
  model.train()
  for i, batch_data in enumerate(tqdm(train_loader)):
      optimizer.zero_grad()
      sents, labels = batch_data
      sents = sents.to(device)
      labels = labels.to(device)

      pred = model(sents['input_ids'].squeeze(1), sents['attention_mask'].squeeze(1), sents['token_type_ids'].squeeze(1))
      loss = loss_func(pred, labels)
      loss.backward()
      optimizer.step()
  
  model.eval()
  with torch.no_grad():
    for i, batch_data in enumerate(tqdm(valid_loader)):
        sents, labels = batch_data
        sents = sents.to(device)
        labels = labels.to(device)

        pred = model(sents['input_ids'].squeeze(1), sents['attention_mask'].squeeze(1), sents['token_type_ids'].squeeze(1))
        acc += torch.eq(pred.argmax(dim=-1).cpu(), labels.cpu()).sum().item()
        tot += 1
  
  print(f'EPOCH_{epoch}',round(acc/tot,5))

  0%|          | 0/306 [00:00<?, ?it/s]

  0%|          | 0/306 [00:00<?, ?it/s]

EPOCH_1 26.99346


  0%|          | 0/306 [00:00<?, ?it/s]

  0%|          | 0/306 [00:00<?, ?it/s]

...

  0%|          | 0/306 [00:00<?, ?it/s]

EPOCH_19 31.86601


  0%|          | 0/306 [00:00<?, ?it/s]

  0%|          | 0/306 [00:00<?, ?it/s]

EPOCH_20 31.94118


---
### 4-4. NLI Model Another Dataset
----

In [3]:
from datasets import load_dataset
sem_eval = load_dataset("sem_eval_2018_task_1",'subtask5.english')

  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
sem_eval

DatasetDict({
    train: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 6838
    })
    test: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 3259
    })
    validation: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 886
    })
})

In [4]:
class BertDataset(Dataset):

  def __init__(self, data, tokenizer):
    self.sentence = []
    self.label = []
    self.tokenizer = tokenizer
    for meta in data:
      self.sentence.append(self.tokenizer(meta['Tweet'], max_length=256, padding='max_length', truncation=True, return_tensors='pt'))
      temp = []
      for key in meta.keys():
        if key in ['ID', 'Tweet']:
          continue
        if meta[key] == True:
          temp.append(1)
        else:
          temp.append(0)
      self.label.append(temp)
 
  def __len__(self):
    return len(self.sentence)

  def __getitem__(self, idx):
    return self.sentence[idx], self.label[idx]

In [5]:
class SentenceClassificationModelMultiClass(nn.Module):
  def __init__(self):
    super(SentenceClassificationModelMultiClass, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    # num of class (11) -> 0, 1 BCE loss
    self.classification_layer = nn.Linear(768, 11)
  
  def forward(self, input_ids, attention_mask, token_type_ids):
    bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    cls_token = bert_output[0][:,0,:]
    pred = self.classification_layer(cls_token)
    pred = F.sigmoid(pred)
    return pred

In [6]:
# multi label classification
train_data = BertDataset(sem_eval['train'], tokenizer)
valid_data = BertDataset(sem_eval['validation'], tokenizer)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last = True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False, drop_last = True)

In [7]:
epochs = 20
model = SentenceClassificationModelMultiClass().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
loss_func = nn.BCELoss()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
for epoch in range(1, epochs+1):
  valid_loss = 0.0
  tot = 0.0
  loss_check = 0.0
  model.train()
  for i, batch_data in enumerate(tqdm(train_loader)):
    optimizer.zero_grad()
    sents, labels = batch_data
    sents = sents.to(device)
    labels = torch.stack(labels, dim = 1).type(torch.float).to(device)

    pred = model(sents['input_ids'].squeeze(1), sents['attention_mask'].squeeze(1), sents['token_type_ids'].squeeze(1))
    loss = loss_func(pred, labels)
    loss.backward()
    loss_check += loss.item()

    optimizer.step()

  print(f'EPOCH_{epoch}_LOSS',loss_check / len(train_loader.dataset))
  model.eval()
  with torch.no_grad():
    for i, batch_data in enumerate(tqdm(valid_loader)):
      sents, labels = batch_data
      sents = sents.to(device)
      labels = torch.stack(labels, dim = 1).type(torch.float).to(device)
      
      pred = model(sents['input_ids'].squeeze(1), sents['attention_mask'].squeeze(1), sents['token_type_ids'].squeeze(1))
      loss = loss_func(pred, labels)   
      valid_loss += loss.item()
      tot += len(sents)

  print(f'EPOCH_{epoch}',round(valid_loss/tot,5))  

  0%|          | 0/213 [00:00<?, ?it/s]

EPOCH_1_LOSS 0.011047390710456777


  0%|          | 0/27 [00:00<?, ?it/s]

EPOCH_1 0.10165


  0%|          | 0/213 [00:00<?, ?it/s]

...

EPOCH_19 0.22047


  0%|          | 0/213 [00:00<?, ?it/s]

EPOCH_20_LOSS 0.0004634136742587809


  0%|          | 0/27 [00:00<?, ?it/s]

EPOCH_20 0.22754
